In [13]:
#imports
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from helpers.products import Products
from directed_graph_generators.directed_tree import DirectedTree
from directed_graph_generators.lattice import DirectedLattice
from directed_graph_generators.directed_stochastic_block_model import DirectedStochasticBlockModel
from directed_graph_generators.directed_erdos_renyi import DirectedErdosRenyi

from clustering_algorithms.exponential_clustering import Exponential_Clustering, Exponential_Clustering_no_evecs
from clustering_algorithms.zanetti_clustering import DSBM_Clustering_Zanetti

from helpers.get_hermitian_adjacency_matrix import get_hermitian_adjacency_matrix
from helpers.deduce_metagraph import deduce_metagraph

In [14]:
# make a 4 cycle directed stochastic block model
F = np.array([[0.5,1.0,0.5,0.0],
              [0.5,0.5,1.0,0.5],
              [0.5,0.5,0.5,1.0],
              [1.0,0.5,0.5,0.5]])
A = DirectedStochasticBlockModel(4,400,1,1,F).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)]]
M = deduce_metagraph(A, clusters, normalize=True)

In [24]:
def lower_bound_and_sum_alpha(A,M,l,i,clusters):
    """ A - adjacency matrix of graph
        M - metagraph of graph
       l - index of last informative eigenvalue/eigenvector
       j - index of eigenvalue/eigenvector we are interested in
       clusters - list of lists of vertices in each cluster. We use this to construct \bar{g}_l
    Returns: 
    norm of difference between lth eigenvector of L_A and L_M
    and \gamma_l / \lambda_{l+1}
    """
    #getting A_herm
    A_herm = get_hermitian_adjacency_matrix(A,normalize=True)

    # getting D to create indicator vectors
    D_half = np.zeros(A_herm.shape)
    A_sym = A + A.T
    np.fill_diagonal(D_half,[np.sqrt(d) for d in np.sum(np.abs(A_sym), axis=1) if d != 0])

    # getting L_A and L_M
    M_herm = get_hermitian_adjacency_matrix(M,normalize=False)
    L_A = np.eye(A.shape[0]) - A_herm
    L_M = np.eye(M.shape[0]) - M_herm

    # compute eigenvectors of L_A and obtain f
    eigvals, eigvecs = np.linalg.eig(L_A)
    eigvals = np.real(eigvals)
    idx_A = eigvals.argsort()
    eigvals = eigvals[idx_A]
    eigvecs = eigvecs[:,idx_A]

    # compute eigenvectors of L_M and obtain g
    meta_eigvals, meta_eigvecs = np.linalg.eig(L_M)
    meta_eigvals = np.real(meta_eigvals)
    idx_M = meta_eigvals.argsort()
    meta_eigvals = meta_eigvals[idx_M]
    meta_eigvecs = meta_eigvecs[:,idx_M]

    # compute \bar{g_i}
    g = meta_eigvecs[:,i-1]
    g_bar_i = np.zeros(len(eigvecs), dtype=np.complex_)

    for j in range(len(clusters)):
        # define complex indicator vector
        indicator = np.zeros(len(eigvecs),dtype=np.complex_)
        indicator[clusters[j]] = 1 + 0j
        indicator = D_half @ indicator
        #normalize
        indicator = indicator/np.linalg.norm(indicator)
        g_bar_i[clusters[j]] = g[j]*indicator[clusters[j]]

    alpha_i = np.zeros(l, dtype=np.complex_)
    
    for j in range(l):
        f_j = eigvecs[:,j]
        alpha_i[j] = np.vdot(f_j,g_bar_i)
    
    sum_alpha_ij_norms = np.linalg.norm(alpha_i)**2
    lower_bound = (eigvals[l] - meta_eigvals[i-1])/(eigvals[l] - eigvals[0])

    return sum_alpha_ij_norms, lower_bound


In [ ]:
def lower_bound_and_sum_beta(A,M,l,clusters):
    """ A - adjacency matrix of graph
        M - metagraph of graph
       l - index of last informative eigenvalue/eigenvector
       j - index of eigenvalue/eigenvector we are interested in
       clusters - list of lists of vertices in each cluster. We use this to construct \bar{g}_l
    Returns: 
    norm of difference between lth eigenvector of L_A and L_M
    and \gamma_l / \lambda_{l+1}
    """
    #getting A_herm
    A_herm = get_hermitian_adjacency_matrix(A,normalize=True)

    # getting D to create indicator vectors
    D_half = np.zeros(A_herm.shape)
    A_sym = A + A.T
    np.fill_diagonal(D_half,[np.sqrt(d) for d in np.sum(np.abs(A_sym), axis=1) if d != 0])

    # getting L_A and L_M
    M_herm = get_hermitian_adjacency_matrix(M,normalize=False)
    L_A = np.eye(A.shape[0]) - A_herm
    L_M = np.eye(M.shape[0]) - M_herm

    # compute eigenvectors of L_A and obtain f
    eigvals, eigvecs = np.linalg.eig(L_A)
    eigvals = np.real(eigvals)
    idx_A = eigvals.argsort()
    eigvals = eigvals[idx_A]
    eigvecs = eigvecs[:,idx_A]

    # compute eigenvectors of L_M and obtain g
    meta_eigvals, meta_eigvecs = np.linalg.eig(L_M)
    meta_eigvals = np.real(meta_eigvals)
    idx_M = meta_eigvals.argsort()
    meta_eigvals = meta_eigvals[idx_M]
    meta_eigvecs = meta_eigvecs[:,idx_M]

    g_bars = []
    for i in range(l):
        # compute \bar{g_i}
        g = meta_eigvecs[:,i]
        g_bar_i = np.zeros(len(eigvecs), dtype=np.complex_)
        for j in range(len(clusters)):
            # define complex indicator vector
            indicator = np.zeros(len(eigvecs),dtype=np.complex_)
            indicator[clusters[j]] = 1 + 0j
            indicator = D_half @ indicator
            #normalize
            indicator = indicator/np.linalg.norm(indicator)
            g_bar_i[clusters[j]] = g[j]*indicator[clusters[j]]
        g_bars.append(g_bar_i)

    beta = np.zeros((l,l), dtype=np.complex_)
    lower_bounds = []
    for i in range(l):
        lower_bound = (eigvals[l] - meta_eigvals[i])/(eigvals[l] - eigvals[0])
        lower_bounds.append(lower_bound)
        for j in range(l):
            beta[i,j] = np.vdot(g_bars[j], eigvecs[i])
    
    
    sum_beta_ij_norms = np.linalg.norm(beta, ord='fro')**2
    lower_bound = np.sum(lower_bounds)
    

    return sum_beta_ij_norms, lower_bound

In [29]:
sum_alpha_ij_norms, lower_bound = lower_bound_and_sum_alpha(A,M,l=1,i=1,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('lower bound: ', lower_bound)
sum_beta_ij_norms, lower_bound = lower_bound_and_sum_beta(A,M,l=1,clusters=clusters)
print('actual sum: ', sum_beta_ij_norms)
print('lower bound: ', lower_bound)

actual sum:  0.9977736773434925
lower bound:  0.8558340711438127


In [6]:
#4 cycle again with smaller p and q
F = np.array([[0.5,1.0,0.5,0.0],
              [0.5,0.5,1.0,0.5],
              [0.5,0.5,0.5,1.0],
              [1.0,0.5,0.5,0.5]])
A = DirectedStochasticBlockModel(4,400,0.2,0.2,F).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)]]
M = deduce_metagraph(A, clusters, normalize=True)

sum_alpha_ij_norms, upper_bound = lower_bound_and_sum_alpha(A,M,l=1,i=1,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('upper bound: ', upper_bound)


actual sum:  0.9748415535779221
upper bound:  0.8851610486488242


In [7]:
#different structure
F = np.array([[0.5,1.0,1.0,1.0],
              [0.0,0.5,1.0,0.5],
              [0.0,0.5,0.5,1.0],
              [0.0,0.5,0.5,0.5]])
A = DirectedStochasticBlockModel(4,400,0.5,0.5,F).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)]]
M = deduce_metagraph(A, clusters, normalize=True)

sum_alpha_ij_norms, upper_bound = lower_bound_and_sum_alpha(A,M,l=1,i=1,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('upper bound: ', upper_bound)

actual sum:  0.9937025933043092
upper bound:  0.9341334795964588


In [8]:
#different structure
F = np.array([[0.5,1.0,1.0,1.0],
              [0.0,0.5,1.0,0.5],
              [0.0,0.5,0.5,1.0],
              [0.0,0.5,0.5,0.5]])
A = DirectedStochasticBlockModel(4,400,0.1,0.1,F).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)]]
M = deduce_metagraph(A, clusters, normalize=True)

sum_alpha_ij_norms, upper_bound = lower_bound_and_sum_alpha(A,M,l=1,i=1,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('upper bound: ', upper_bound)

actual sum:  0.9471179716314194
upper bound:  0.7132006405131784


In [9]:
F_3 = np.array([[0.5, 1.0, 0.0, 0.5, 0.5, 0.5], 
                [0.0, 0.5, 1.0, 0.5, 0.5, 0.5], 
                [1.0, 0.0, 0.5, 1.0, 0.5, 0.5],
                [0.5, 0.5, 0.0, 0.5, 1.0, 0.0],
                [0.5, 0.5, 0.5, 0.0, 0.5, 1.0],
                [0.5, 0.5, 0.5, 1.0, 0.0, 0.5]])
A = DirectedStochasticBlockModel(6,600,0.5,0.5,F_3).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)], [i for i in range(400,500)], [i for i in range(500,600)]]
M = deduce_metagraph(A, clusters, normalize=True)

sum_alpha_ij_norms, upper_bound = lower_bound_and_sum_alpha(A,M,l=3,i=2,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('upper bound: ', upper_bound)


actual sum:  0.9800372547673044
upper bound:  0.4591270934617292


In [10]:
F_3 = np.array([[0.5, 1.0, 0.0, 0.5, 0.5, 0.5], 
                [0.0, 0.5, 1.0, 0.5, 0.5, 0.5], 
                [1.0, 0.0, 0.5, 1.0, 0.5, 0.5],
                [0.5, 0.5, 0.0, 0.5, 1.0, 0.0],
                [0.5, 0.5, 0.5, 0.0, 0.5, 1.0],
                [0.5, 0.5, 0.5, 1.0, 0.0, 0.5]])
A = DirectedStochasticBlockModel(6,600,0.1,0.1,F_3).adjacency_matrix
clusters = [[i for i in range(100)], [i for i in range(100,200)], [i for i in range(200,300)], [i for i in range(300,400)], [i for i in range(400,500)], [i for i in range(500,600)]]
M = deduce_metagraph(A, clusters, normalize=True)

sum_alpha_ij_norms, upper_bound = lower_bound_and_sum_alpha(A,M,l=3,i=1,clusters=clusters)
print('actual sum: ', sum_alpha_ij_norms)
print('upper bound: ', upper_bound)

actual sum:  0.9336753510166479
upper bound:  0.6401921426759368
